In [9]:
import gradio as gr
import numpy as np
import cv2
from ultralytics import YOLO
from paddleocr import TextRecognition, PaddleOCR
from PIL import Image


# --- Cargar modelos ---
yolo_model = YOLO("/home/gnz/GitHub/yolo11_container/runs/detect/train2/weights/best.pt")
ocr_model = PaddleOCR(lang='en', use_textline_orientation=True)


Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/gnz/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/gnz/.paddlex/official_models/UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/gnz/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/gnz/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('en_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/home/gnz/.paddlex/official_models/en_PP-OCR

In [ ]:
def process_image(image):

    img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = yolo_model.predict(img, conf=0.25, verbose=False)

    annotated = results[0].plot()
    crops, texts = [], []

    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        crop = img[y1:y2, x1:x2]

        # --- Convertir crop a PIL para TextRecognition ---
        crop_pil = Image.fromarray(crop.astype(np.uint8))

        # --- OCR ---
        ocr_result = ocr_model.predict(crop_pil)
        texts_in_crop = [res['text'] for res in ocr_result if res.get('text')]

        crops.append(crop)
        texts.append(" ".join(texts_in_crop) if texts_in_crop else "(sin texto)")

    return annotated, crops, "\n".join(texts)


# --- Interfaz Gradio ---
with gr.Blocks() as demo:
    gr.Markdown("## 🚢 Detección de IDs y OCR en Contenedores (Inglés)")

    with gr.Row():
        inp_img = gr.Image(type="numpy", label="Sube una imagen")
        out_img = gr.Image(type="numpy", label="Imagen anotada")
    gallery = gr.Gallery(label="Crops detectados", columns=3, height="auto")
    out_texts = gr.Textbox(label="Resultados OCR", lines=6)

    inp_img.change(process_image, inputs=inp_img, outputs=[out_img, gallery, out_texts])

demo.launch()

* Running on local URL:  http://127.0.0.1:7878
* To create a public link, set `share=True` in `launch()`.


Not supported input data type! Only `numpy.ndarray` and `str` are supported! So has been ignored: <PIL.Image.Image image mode=RGB size=125x64 at 0x7F7C698D2780>.
Not supported input data type! Only `numpy.ndarray` and `str` are supported! So has been ignored: <PIL.Image.Image image mode=RGB size=139x64 at 0x7F7C69E6E030>.
Not supported input data type! Only `numpy.ndarray` and `str` are supported! So has been ignored: <PIL.Image.Image image mode=RGB size=274x69 at 0x7F7C203A2930>.
